Import necessary packages

In [789]:
import pandas as pd
from datetime import datetime


Import given .csv files

In [790]:
reference_fields_df = pd.read_csv('reference_fileds.csv')
reference_securities_df = pd.read_csv('reference_securities.csv')

Import the given .dif file

In [791]:
with open('corp_pfd.dif') as file:
    corp_pfd = file.readlines()

Save an array of the file names that will be used. Per the instructions, this will be between the 'START-OF-FIELDS' and 'END-OF-FIELDS' string

### Validation

I will be first be examining the files to make sure that my logic to parse the code will work as expected

In [792]:
#Check the column names and notice the "\n"
corp_pfd[0:5]

['START-OF-FILE\n',
 'PROGRAMNAME=getdata\n',
 'DATEFORMAT=yyyymmdd\n',
 '\n',
 'START-OF-FIELDS\n']

I will strip the new line character for determing column names, and will save the changes in a new list

In [793]:
corp_pfd_stripped = []
for x in corp_pfd:
    #Validate the new line character only occurs once per list item
    if x.count('/n') > 1:
        print('ERROR: ', x, ' has more than 1 new line character')
    
    corp_pfd_stripped.append(x.rstrip('\n'))

Validate that the 'START-OF-FIELDS' and 'END-OF-FIELDS' strings only exist once to avoid any code issues (both need to return 1)


In [794]:
print(corp_pfd_stripped.count('START-OF-FIELDS'))
print(corp_pfd_stripped.count('END-OF-FIELDS'))

1
1


Extract the column names into the list
'col_names'

In [795]:
#Determine where the column fields are located
col_name_start = corp_pfd_stripped.index('START-OF-FIELDS')
col_name_end = corp_pfd_stripped.index('END-OF-FIELDS')
col_names = []
for idx, col in enumerate(corp_pfd_stripped):
    #ignore comments and new lines, as they are not relevant
    if idx > col_name_start and idx < col_name_end and len(col) != 0 and col[0] != '#':
        col_names.append(col)

We will now read in the data for step 1

### Step 1

In [796]:
data_start = corp_pfd.index('START-OF-DATA\n') + 1
data_end = corp_pfd.index('END-OF-DATA\n') - 1
data_list = corp_pfd[data_start:data_end]

In [797]:
#Any FutureWarnings are unnecesary for this purpose, adding for quicker performance
import warnings
warnings.filterwarnings("ignore", category = FutureWarning)

#parse data by the '|' delimiter
test = data_list[0].split('|')
step_1_df = pd.DataFrame()
for data in data_list:
    split_data = data.split('|')
    #remove the \n that signifies the end of the data line
    split_data = split_data[:len(split_data) - 1]
    #Convert to series so that it can be appended
    split_data_df = pd.Series(split_data)
    step_1_df = step_1_df.append(split_data_df, ignore_index = True)
    
#Name columns that we determined above when extracting the column names
step_1_df.columns = col_names




### Step 2

In [798]:
reference_fields_uppercase = []
reference_fields_df
for x in reference_fields_df['field']:
    #If the column isn't in the new list, we'll skip it
    if x.upper()in col_names:
        #Add all the columns from the reference field as uppercase values
        reference_fields_uppercase.append(x.upper())
#Trim our dataframe so that it has only the columns found in the 'reference_fields.csv' file
step_2_df = step_1_df[reference_fields_uppercase]

### Step 3 part 1

In [799]:
#For part 1 we will identify the securities that are not found in reference_securities_csv (part 2 will format this dataframe)

new_securities = []

for idx, x in step_2_df.iterrows():
    #If a securitity exists in our parsed data, but not the reference_security file, we will save the id_bb_global
    if not reference_securities_df['id_bb_global'].str.contains(x['ID_BB_GLOBAL']).any():
        new_securities.append(x['ID_BB_GLOBAL'])



## Step 3 part 2

Find the rows for the ID_BB_GLOBAL values that are found in the input file but not in the reference securities csv

In [800]:
step_3_df = step_2_df[step_2_df['ID_BB_GLOBAL'].isin(new_securities)]

Change the column names so that they match the columns of reference_securities.csv


In [801]:
reference_securities_df_upper = []
for x in reference_securities_df.columns:
    #reference_securities columns are lowercase, we will make them uppercase to make them easier to compare
    reference_securities_df_upper.append(x.upper())
step_3_df = pd.DataFrame(step_1_df, columns = reference_securities_df_upper)



In [802]:
print(step_3_df)

      ID_BB_GLOBAL       ID_ISIN   ID_CUSIP ID_SEDOL1  TICKER  \
0     BBG00KRL8RW6  HK0000416609  AS3629704   BJGV5B5    MTRC   
1     BBG00KRMSZ63  ARCILB560016  AS3764410            CILBAR   
2     BBG00NZ74Q67  XS1989092116  ZS2774298   BJK3NS0  REDPRO   
3     BBG00000F4P6  CA4590568X88  4590568X8              IBRD   
4     BBG009HTFZR5  US400489AH37  400489AH3   BYY0995  POSADA   
...            ...           ...        ...       ...     ...   
2890  BBG00P8S59R0  XS2004865957  ZS8316854              ITAU   
2891  BBG00P8SNLL9  US90346JAC62  90346JAC6            USJACU   
2892  BBG00P8SXNZ9  USP9634CAC57  ZS8402340   BJYJF77  USJACU   
2893  BBG00P96HDD0                ZS8705577             LTMCI   
2894  BBG00P9C1MJ8                ZS8952088            SECUCL   

                         NAME     EXCH_CODE                   ISSUER  \
0             MTR CORP CI LTD    NOT LISTED          MTR CORP CI LTD   
1                CILBRAKE SRL  BUENOS AIRES             CILBRAKE SRL   
2  

Only include rows that are included in the input file, but not in reference_securities.csv

In [803]:
step_3_df = step_1_df[step_1_df['ID_BB_GLOBAL'].isin(new_securities)]



In [804]:
print(step_3_df)

     TICKER_YELLOW_KEY ST1 REF1  TICKER       CPN  MATURITY SERIES  \
1446    ED5429110 Corp   0  210  OREKON  3.500000  20281201          

                    NAME   SHORT_NAME ISSUER_INDUSTRY  ... INDUSTRY_GROUP_NUM  \
1446  ORESUNDSKONSORTIET  ORESUNDSKON   GOVT REGIONAL  ...              20075   

     INDUSTRY_SECTOR_NUM ISSUERS_STOCK INFLATION_LAG MAKE_WHOLE_CALL_SPREAD  \
1446               10010                           3                   N.A.   

             ISSUER_BULK    ID_BB_SEC_NUM_DES FEED_SOURCE  \
1446  ORESUNDSKONSORTIET  OREKON 3.5 12/01/28         BGN   

     ID_BB_GLOBAL_COMPANY ID_BB_GLOBAL_COMPANY_NAME  
1446         BBG001FKN3Y7    Oresundsbro Konsortiet  

[1 rows x 213 columns]


Change all the columns to the reference_security columns in uppercase

In [805]:
step_3_correct_columns = pd.DataFrame(step_3_df, columns = reference_securities_df_upper)

Lowercase the column names

In [806]:
step_3_correct_columns.columns = step_3_correct_columns.columns.str.lower()

Create new_securities.csv

In [807]:
step_3_correct_columns.to_csv('new_securities.csv', sep=',', index = False)

### Step 4

Export to csv

In [808]:
step_3_correct_columns.to_csv('step_4_output_part_1.csv', sep=',', index = False)

Count is only here to save time, if you would like to valide that it exports a complete csv, delete the:'if count > 200: break' line
The outputs shared in the git repository are from only the first 200 rows

In [809]:
step4 = pd.DataFrame(columns = ['ID_BB_GLOBAL','FIELD','VALUE', 'SOURCE','TSTAMP'])
count = 0
for idx, x in step_1_df.iterrows():
    #
#     count = count + 1
#     if count > 200:
#         break
    for y in reference_fields_df['field']:
        timestamp = datetime.now()
        printed_time = timestamp.strftime("%d/%m/%Y %H:%M:%S")
        try:
            appendRow = {
              'ID_BB_GLOBAL': x['ID_BB_GLOBAL'],
              'FIELD': y,
              'VALUE': x[y],
              'SOURCE':'corp_pdf.dif',
              'TSTAMP': printed_time,  
              }
            step4 = step4.append(appendRow, ignore_index=True)
    
        except:
            appendRow = {
              'ID_BB_GLOBAL': x['ID_BB_GLOBAL'],
              'FIELD': y,
              'VALUE': 'NA',
              'SOURCE':'corp_pdf.dif',
              'TSTAMP': printed_time,  
              }
            step4 = step4.append(appendRow, ignore_index=True)

In [810]:
step4.to_csv('step_4_output_part_2.csv', sep=',', index = False)